In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from pathlib import Path
from selenium import webdriver
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
import plotly.express as px
import plotly.graph_objects as go


/Users/francip/Desktop/Master/PRVD/PRVD-1/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Análisis del precio de la vivienda en Alicante (centro)

In [30]:
def get_html(file):
    with open(file) as f:
        html = f.read()

    return BeautifulSoup(html, 'html.parser')

def extract_values(soup):
    table_container = soup.find('div', {'class': 'table__scroll'})
    rows = table_container.findAll('tr')

    ## Saltarse la primera fila que son los headers
    results = []
    for row in rows[1:]:
        tds = row.findAll('td')

        tds = [td.text.strip() if td.text.strip() != 'n.d.' else None for td in tds]

        row_result = {
            'Mes': tds[0],
            'Precio m2': tds[1],
            'Variación mensual': tds[2],
            'Variación trimestral': tds[3],
            'Variación anual': tds[4]
        }
        
        results.append(row_result)

    df = pd.DataFrame(results)

    df['Fecha'] = pd.to_datetime(df['Mes'], format='%B %Y', errors='coerce')
    df = df.sort_values(by='Fecha', ascending=True)

    ## Quitar el €/m2 y el . de la columna Precio m2
    df['Precio m2'] = df['Precio m2'].replace({r'[^\d.,]': ''}, regex=True).replace('\.', '', regex=True).astype(float) / 10

    ## Borrar las filas que no tengan datos de Precio m2
    df = df[~pd.isna(df['Precio m2'])]

    ## Quitar el % de las columnas de Variación y pasarlo a float
    df[['Variación mensual', 'Variación trimestral', 'Variación anual']] = \
        df[['Variación mensual', 'Variación trimestral', 'Variación anual']].apply(
            lambda x: x.replace('None', '0').replace('%', '', regex=True).replace('\s+', '', regex=True).replace(',', '.', regex=True).fillna(0).astype(float))

    ## Añadir 3 nuevas columnas con las variaciones
    df['Variación mensual €'] = df['Precio m2'] * (1 + df['Variación mensual'] / 100)
    df['Variación trimestral €'] = df['Precio m2'] * (1 + df['Variación trimestral'] / 100)
    df['Variación anual €'] = df['Precio m2'] * (1 + df['Variación anual'] / 100)

    return df


In [31]:
soup = get_html('htmls/historico_alicante_venta.html')
df_historico_precios_venta_alicante = extract_values(soup)

In [ ]:
# ## URL: https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/comunitat-valenciana/alicante-alacant/alicante-alacant/centro/historico/
# with open('htmls/historico_alicante_venta.html') as f:
#     html = f.read()

# soup = BeautifulSoup(html, 'html.parser')

# table_container = soup.find('div', {'class': 'table__scroll'})
# rows = table_container.findAll('tr')

# ## Saltarse la primera fila que son los headers
# results = []
# for row in rows[1:]:
#     tds = row.findAll('td')

#     tds = [td.text.strip() if td.text.strip() != 'n.d.' else None for td in tds]

#     row_result = {
#         'Mes': tds[0],
#         'Precio m2': tds[1],
#         'Variación mensual': tds[2],
#         'Variación trimestral': tds[3],
#         'Variación anual': tds[4]
#     }
    
#     results.append(row_result)

# df_historico_precios_venta_alicante = pd.DataFrame(results)

# ## Crear una nueva columna Fecha para poder ordenar de forma ascendente
# df_historico_precios_venta_alicante['Fecha'] = pd.to_datetime(df_historico_precios_venta_alicante['Mes'], format='%B %Y', errors='coerce')
# df_historico_precios_venta_alicante = df_historico_precios_venta_alicante.sort_values(by='Fecha', ascending=True)

# ## Quitar el €/m2 y el . de la columna Precio m2
# df_historico_precios_venta_alicante['Precio m2'] = df_historico_precios_venta_alicante['Precio m2'].replace({r'[^\d.,]': ''}, regex=True).replace('\.', '', regex=True).astype(float) / 10

# ## Borrar las filas que no tengan datos de Precio m2
# df_historico_precios_venta_alicante = df_historico_precios_venta_alicante[~pd.isna(df_historico_precios_venta_alicante['Precio m2'])]

# ## Quitar el % de las columnas de Variación y pasarlo a float
# df_historico_precios_venta_alicante[['Variación mensual', 'Variación trimestral', 'Variación anual']] = \
#     df_historico_precios_venta_alicante[['Variación mensual', 'Variación trimestral', 'Variación anual']].apply(
#         lambda x: x.replace('None', '0').replace('%', '', regex=True).replace('\s+', '', regex=True).replace(',', '.', regex=True).fillna(0).astype(float))

# ## Añadir 3 nuevas columnas con las variaciones
# df_historico_precios_venta_alicante['Variación mensual €'] = df_historico_precios_venta_alicante['Precio m2'] * (1 + df_historico_precios_venta_alicante['Variación mensual'] / 100)
# df_historico_precios_venta_alicante['Variación trimestral €'] = df_historico_precios_venta_alicante['Precio m2'] * (1 + df_historico_precios_venta_alicante['Variación trimestral'] / 100)
# df_historico_precios_venta_alicante['Variación anual €'] = df_historico_precios_venta_alicante['Precio m2'] * (1 + df_historico_precios_venta_alicante['Variación anual'] / 100)


In [26]:
fig = go.Figure()

fig.add_trace(
    go.Bar(x=df_historico_precios_venta_alicante['Mes'],
           y=df_historico_precios_venta_alicante['Precio m2'],
           name="Precio m2")
)

fig.add_trace(
    go.Scatter(x=df_historico_precios_venta_alicante['Mes'],
               y=df_historico_precios_venta_alicante['Variación mensual €'],
               mode='lines',
               name="Variación mensual",
               hovertemplate=(
                   'Mes: %{x}<br>'
                   'Variación mensual: %{y:.3f} €<br>' 
                   'Porcentaje: %{text}%<br>' 
               ),
               text=df_historico_precios_venta_alicante['Variación trimestral'].apply(lambda x: f"{x:.2f}"),
               line=dict(color='red'))
)

fig.add_trace(
    go.Scatter(x=df_historico_precios_venta_alicante['Mes'],
               y=df_historico_precios_venta_alicante['Variación trimestral €'],
               mode='lines',
               name="Variación trimestral",
               hovertemplate=(
                   'Mes: %{x}<br>'
                   'Variación trimestral: %{y:.3f} €<br>' 
                   'Porcentaje: %{text}%<br>' 
               ),
               text=df_historico_precios_venta_alicante['Variación trimestral'].apply(lambda x: f"{x:.2f}"),
               line=dict(color='green'))
)

fig.add_trace(
    go.Scatter(x=df_historico_precios_venta_alicante['Mes'],
               y=df_historico_precios_venta_alicante['Variación anual €'],
               mode='lines',
               name="Variación anual",
               hovertemplate=(
                   'Mes: %{x}<br>'
                   'Variación anual: %{y:.3f} €<br>' 
                   'Porcentaje: %{text}%<br>' 
               ),
               text=df_historico_precios_venta_alicante['Variación anual'].apply(lambda x: f"{x:.2f}"),
               line=dict(color='purple'))
)

fig.update_layout(
    title='Evolución del precio por m2 de la vivienda en Alicante (centro)',
    xaxis_title="Mes",
    yaxis_title="Precio por m2",
    yaxis2=dict(
        title="Variación mensual (%)",
        overlaying='y',
        side='right'
    )
)

fig.show()

In [19]:
## URL: https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/alquiler/comunitat-valenciana/alicante-alacant/alicante-alacant/centro/
with open('htmls/historico_alicante_alquiler.html') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

table_container = soup.find('div', {'class': 'table__scroll'})
rows = table_container.findAll('tr')

## Saltarse la primera fila que son los headers
results = []
for row in rows[1:]:
    tds = row.findAll('td')

    tds = [td.text.strip() if td.text.strip() != 'n.d.' else None for td in tds]

    row_result = {
        'Mes': tds[0],
        'Precio m2': tds[1],
        'Variación mensual': tds[2],
        'Variación trimestral': tds[3],
        'Variación anual': tds[4]
    }
    
    results.append(row_result)

df_historico_precios_alquiler_alicante = pd.DataFrame(results)